Import all necessary libraries.

In [ ]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister, Parameter
from qiskit.quantum_info import Statevector, Operator

import matplotlib.pyplot as plt
import numpy as np

First consider the case where n=2, then we are dealing with a 4-4 sudoku puzzle here.
For the purpose of testing, let the puzzle be (generated by AI):
[1, 0, 0, 4]
[0, 3, 0, 0] 
[0, 0, 1, 0]
[2, 0, 0, 3],
where the cells that show 0 are the cells not filled-in.

In [ ]:
puzzle = np.array([
        [1, 0, 0, 4],
        [0, 3, 0, 0], 
        [0, 0, 1, 0],
        [2, 0, 0, 3]
    ])
print(puzzle)
pzl_flt = puzzle.flatten()
print(pzl_flt)

Initialize the puzzle in quantum circuit using classical approach.
The puzzle_Qinit method takes the flattened puzzle as input and outputs an array of the positions of empty cells.
Treating the whole puzzle as a quantum state, we have potentially 4^16 number of states, which is 2^32, and hence we need 32 qubits to represent every possible state.
For the encoding of the values, I choose the order with the tensor product, for the consideration that the built-in statevector function might be used somewhere and this is more consistent.
Encode:
value 1 is represented by |00>
value 2 is represented by |10>
value 3 is represented by |01>
value 4 is represented by |11>
value 0 means it is empty

In [ ]:
def puzzle_qinit(pzlFlt):
    # construct a 32-qubit register all initialized to |0>
    qReg = QuantumRegister(32, "puzzle")
    qPuzzle = QuantumCircuit(qReg)
    # initialize circuit index as 0, since every cell has 4 states, we process two qubits at a time.
    cirIdx = 0
    for value in pzlFlt:
        if value == 1:
            # nothing needs to be done since |00> already represents 1
            pass
        elif value == 2:
            # flip the first bit to get |10>
            qPuzzle.x(qReg[cirIdx])
        elif value == 3:
            # flip the second bit to get |01>
            qPuzzle.x(qReg[cirIdx+1])
        elif value == 4:
            # flip both bits to get |11>
            qPuzzle.x(qReg[cirIdx])
            qPuzzle.x(qReg[cirIdx+1])
        elif value == 0:
            # apply Hadamard gate to empty cell bits
            qPuzzle.h(qReg[cirIdx])
            qPuzzle.h(qReg[cirIdx+1])
        # no else for error handling for simplicity at the moment
        cirIdx = cirIdx + 2
    return qPuzzle

q_puzzle = puzzle_qinit(pzl_flt)
q_puzzle.draw(output="mpl", style="bw")

Next up to construct the Oracle:
For the "f" in the Oracle, I plan to map the solved sudoku as 1 and otherwise 0

In [ ]:
def oracle_f()